In [196]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import math
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список bronze S3

In [75]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)

In [76]:
bucket = s3.Bucket('bronze')
for obj in bucket.objects.filter(Prefix='hh/vacancies/date=2025-12-02/'):
    print(f"Key: {obj.key}, Last Modified: {obj.last_modified}")

Key: hh/vacancies/date=2025-12-02/part_0001.jsonl.gz, Last Modified: 2025-12-04 11:33:04.024000+00:00
Key: hh/vacancies/date=2025-12-02/part_0002.jsonl.gz, Last Modified: 2025-12-04 11:33:16.993000+00:00
Key: hh/vacancies/date=2025-12-02/part_0003.jsonl.gz, Last Modified: 2025-12-04 11:33:28.421000+00:00
Key: hh/vacancies/date=2025-12-02/part_0004.jsonl.gz, Last Modified: 2025-12-04 11:33:42.325000+00:00
Key: hh/vacancies/date=2025-12-02/part_0005.jsonl.gz, Last Modified: 2025-12-04 11:33:54.827000+00:00
Key: hh/vacancies/date=2025-12-02/part_0006.jsonl.gz, Last Modified: 2025-12-04 11:34:05.566000+00:00
Key: hh/vacancies/date=2025-12-02/part_0007.jsonl.gz, Last Modified: 2025-12-04 11:34:16.584000+00:00
Key: hh/vacancies/date=2025-12-02/part_0008.jsonl.gz, Last Modified: 2025-12-04 11:34:27.901000+00:00
Key: hh/vacancies/date=2025-12-02/part_0009.jsonl.gz, Last Modified: 2025-12-04 11:34:39.896000+00:00
Key: hh/vacancies/date=2025-12-02/part_0010.jsonl.gz, Last Modified: 2025-12-04 11

# Проверка дублей vacancy id

In [79]:
bucket = s3.Bucket('bronze')
bronze_report_dates = set()
for f in bucket.objects.all():
    s3fp = f.key
    dt_start_index = s3fp.index('date=')+len('date=')
    dt_end_index = dt_start_index+len('2025-12-02')
    bronze_report_dates.add(s3fp[dt_start_index:dt_end_index])
bronze_report_dates

{'2025-11-10',
 '2025-11-17',
 '2025-11-18',
 '2025-12-02',
 '2025-12-03',
 '2025-12-04'}

In [156]:
for report_dt in sorted(bronze_report_dates):
    print(report_dt)
    df_lazy = pl.scan_ndjson(
        f"s3://bronze/hh/vacancies/date={report_dt}/*.jsonl.gz",
        storage_options=storage_options,
        infer_schema_length=None
    )
    total_id_cnt = pl.col('id').count()
    unique_id_cnt = pl.col('id').unique().count()
    display(
        df_lazy.select(
            total_id_cnt.alias('total_id_cnt'),
            unique_id_cnt.alias('unique_id_cnt'),
            ((total_id_cnt-unique_id_cnt)/total_id_cnt*100).round(2).alias('share_of_duplicates_pcnt')
        ).collect()
    )
    print()

2025-11-10


total_id_cnt,unique_id_cnt,share_of_duplicates_pcnt
u32,u32,f64
42845,33643,21.48



2025-11-17


total_id_cnt,unique_id_cnt,share_of_duplicates_pcnt
u32,u32,f64
46563,36691,21.2



2025-11-18


...

total_id_cnt,unique_id_cnt,share_of_duplicates_pcnt
u32,u32,f64
233167,107500,53.9



2025-12-03


total_id_cnt,unique_id_cnt,share_of_duplicates_pcnt
u32,u32,f64
257124,126201,50.92



2025-12-04


total_id_cnt,unique_id_cnt,share_of_duplicates_pcnt
u32,u32,f64
291400,136526,53.15


# Сверка вакансий по одинаковому id

In [150]:
for report_dt in sorted(bronze_report_dates):
    print(report_dt)
    df_lazy = pl.scan_ndjson(
        f"s3://bronze/hh/vacancies/date={report_dt}/*.jsonl.gz",
        storage_options=storage_options,
        # infer_schema_length=None,
        schema=vacancy_schema
    )
    vacancy_id = df_lazy.select('id').group_by('id').len().sort(by='len', descending=True).head(1).select('id').collect().item()
    unique_rows_cnt = df_lazy.filter(pl.col('id') == vacancy_id).collect().n_unique()
    print("Кол-во уникальных строк с одинаковым vacancy id", unique_rows_cnt)
    print()

2025-11-10
Кол-во уникальных строк с одинаковым vacancy id 1

2025-11-17
Кол-во уникальных строк с одинаковым vacancy id 1

2025-11-18
Кол-во уникальных строк с одинаковым vacancy id 1

2025-12-02
Кол-во уникальных строк с одинаковым vacancy id 1

2025-12-03
Кол-во уникальных строк с одинаковым vacancy id 1

2025-12-04
Кол-во уникальных строк с одинаковым vacancy id 1



**Алгоритм bronze_hh_loader_dag.py/bronze_hh_loader_v3 запрашивает 20-50% вакансий повторно**

# Проверка корректности query params запросов из bronze_hh_loader_dag.py/bronze_hh_loader_v3

In [157]:
# [2025-12-05 14:36:26] INFO - [DRILL DOWN] Interval 2025-12-04 10:09:13.250000+03:00 - 2025-12-04 10:10:09.500000+03:00 has 2541 items. Drilling by params... source=airflow.task loc=bronze_hh_loader_dag.py:269

In [158]:
import requests

def get(url: str, params: dict = {}, headers: dict = {}):
    headers = {**headers, "HH-User-Agent": "Skill Lens/1.0 (loveyousomuch554@gmail.com)"}
    params = {**params}
    res = requests.get(url=url, params=params, headers=headers)
    res.raise_for_status()
    return res.json()

In [162]:
# ISO 8601 (YYYY-MM-DD) или с точностью до секунды YYYY-MM-DDThh:mm:ss±hhmm
date_from = "2025-12-04T10:09:13+0300"
date_to = "2025-12-04T10:10:09+0300"
# number <= 100, default 10
per_page = 100
# ?x= query string
query_params = {
    "date_from": date_from,
    "date_to": date_to,
    "per_page": per_page,
}
vacancies = get("https://api.hh.ru/vacancies", params=query_params)
vacancies.get('found')

2525

In [163]:
EMPLOYMENT_FORMS = [
    "FULL",  # Полная занятость
    "PART",  # Частичная занятость
    "PROJECT",  # Проектная работа
    "FLY_IN_FLY_OUT",  # Вахта
    "SIDE_JOB",  # Подработка
]

WORK_FORMATS = [
    "ON_SITE",  # На месте
    "REMOTE",  # Удаленно
    "HYBRID",  # Гибрид
    "FIELD_WORK",  # Разъездной
]

In [201]:
%%time
for emp in EMPLOYMENT_FORMS:
    emp_ids = list()
    new_query_params = {
        "date_from": date_from,
        "date_to": date_to,
        "per_page": per_page,
        "employment_form": emp,
    }
    vacancies = get("https://api.hh.ru/vacancies", params=new_query_params)
    print(emp, vacancies.get('found'))
    emp_ids.extend([item['id'] for item in vacancies.get('items', [])])
    found = vacancies.get('found')
    for page in range(1, math.ceil(found/per_page)):
        try:
            vacancies = get("https://api.hh.ru/vacancies", params={'page': page, **new_query_params})
            emp_ids.extend([item['id'] for item in vacancies.get('items', [])])
        except Exception as e:
            print(e)
        finally:
            time.sleep(.5)
    print(emp, "total cnt", len(emp_ids), "unique ids count", len(set(emp_ids)))
    time.sleep(1)

FULL 2042
400 Client Error: Bad Request for url: https://api.hh.ru/vacancies?page=20&date_from=2025-12-04T10%3A09%3A13%2B0300&date_to=2025-12-04T10%3A10%3A09%2B0300&per_page=100&employment_form=FULL
FULL total cnt 2000 unique ids count 2000
PART 87
PART total cnt 87 unique ids count 87
PROJECT 9
PROJECT total cnt 9 unique ids count 9
FLY_IN_FLY_OUT 387
FLY_IN_FLY_OUT total cnt 387 unique ids count 387
SIDE_JOB 9
SIDE_JOB total cnt 9 unique ids count 9
CPU times: user 1.08 s, sys: 159 ms, total: 1.24 s
Wall time: 29.2 s


In [202]:
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

In [203]:
vacancies.get('alternate_url')

'https://hh.ru/search/vacancy?date_from=04.12.2025+10%3A09%3A13&date_to=04.12.2025+10%3A10%3A09&employment_form=PROJECT&enable_snippets=true&items_on_page=100'

In [204]:
%%time
for work in WORK_FORMATS:
    work_ids = list()
    new_query_params = {
        "date_from": date_from,
        "date_to": date_to,
        "per_page": per_page,
        "work_format": work,
    }
    vacancies = get("https://api.hh.ru/vacancies", params=new_query_params)
    print(work, vacancies.get('found'))
    work_ids.extend([item['id'] for item in vacancies.get('items', [])])
    found = vacancies.get('found')
    for page in range(1, math.ceil(found/per_page)):
        try:
            vacancies = get("https://api.hh.ru/vacancies", params={'page': page, **new_query_params})
            work_ids.extend([item['id'] for item in vacancies.get('items', [])])
        except Exception as e:
            print(e)
        finally:
            time.sleep(.5)
    print(work, "total cnt", len(work_ids), "unique ids count", len(set(work_ids)))
    time.sleep(1)

ON_SITE 1187
ON_SITE total cnt 1187 unique ids count 1187
REMOTE 116
REMOTE total cnt 116 unique ids count 116
HYBRID 72
HYBRID total cnt 72 unique ids count 72
FIELD_WORK 274
FIELD_WORK total cnt 274 unique ids count 274
CPU times: user 634 ms, sys: 93.3 ms, total: 727 ms
Wall time: 20 s


In [205]:
vacancies.get('alternate_url')

'https://hh.ru/search/vacancy?date_from=04.12.2025+10%3A09%3A13&date_to=04.12.2025+10%3A10%3A09&enable_snippets=true&items_on_page=100&page=2&work_format=FIELD_WORK'

# Ошибка конкатенации схемы, схемы приходят разные

In [6]:
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [21]:
file_paths = []
for obj in bucket.objects.filter(Prefix='hh/vacancies/date=2025-12-02/'):
     # Формируем полный s3 путь
     file_paths.append(f"s3://bronze/{obj.key}")
lazy_dfs = [
    pl.scan_ndjson(path, storage_options=storage_options, infer_schema_length=None) 
    for path in file_paths
]
# Объединяем их. Polars попытается привести типы (например, Null и String станут String)
combined_lazy = pl.concat(lazy_dfs, how="diagonal") # diagonal позволяет объединять даже если где-то не хватает колонок

In [22]:
print(combined_lazy.count().collect())

InvalidOperationError: 'union'/'concat' inputs should all have the same schema,got
Schema:
name: response_letter_required, field: Boolean
name: salary_range, field: Struct({'from': Int64, 'to': Int64, 'currency': String, 'gross': Boolean, 'mode': Struct({'id': String, 'name': String}), 'frequency': Struct({'id': String, 'name': String})})
name: area, field: Struct({'id': String, 'name': String, 'url': String})
name: published_at, field: String
name: created_at, field: String
name: brand_snippet, field: Struct({'logo': Null, 'logo_xs': Null, 'logo_scalable': Struct({'default': Struct({'width': Int64, 'height': Int64, 'url': String}), 'xs': Struct({'width': Int64, 'height': Int64, 'url': String})}), 'picture': Null, 'picture_xs': Null, 'picture_scalable': Struct({'default': Struct({'width': Int64, 'height': Int64, 'url': String}), 'xs': Struct({'width': Int64, 'height': Int64, 'url': String})}), 'background': Struct({'color': String, 'gradient': Struct({'angle': Float64, 'color_list': Li

In [35]:
schemas = list()

In [36]:
for obj in bucket.objects.filter(Prefix='hh/vacancies/date=2025-12-02/'):
    bucket_path = obj.key
    print(bucket_path)
    df_lazy = pl.scan_ndjson(
        f"s3://bronze/{bucket_path}",
        storage_options=storage_options,
        infer_schema_length=None
    )
    columns = df_lazy.collect_schema().names()
    schemas.append(columns)
    print(len(columns))
    print()

hh/vacancies/date=2025-12-02/part_0001.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0002.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0003.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0004.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0005.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0006.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0007.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0008.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0009.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0010.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0011.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0012.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0013.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0014.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0015.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0016.jsonl.gz
48

hh/vacancies/date=2025-12-02/part_0017.jsonl.gz
49

hh/vacancies/date=2025-12-02/part_0018.jsonl.gz
49

hh/vacancies/date=2025-12-02/part_0019.jsonl.gz
49

hh/vacancies

In [39]:
union_schema = set(c for sc in schemas for c in sc)
len(union_schema)

49

In [44]:
for sch in schemas:
    sch = set(sch)
    for ucol in union_schema:
        if ucol not in sch:
            print(ucol)

immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url
immediate_redirect_url


# Определяем схему и забираем только эти поля, делаем варнинг если появятся новые поля

In [46]:
df_lazy = pl.scan_ndjson(
    f"s3://bronze/hh/vacancies/date=2025-12-02/part_0022.jsonl.gz",
    storage_options=storage_options,
    infer_schema_length=None
)
df_lazy.collect_schema()

Schema([('response_letter_required', Boolean),
        ('salary_range',
         Struct({'from': Int64, 'to': Int64, 'currency': String, 'gross': Boolean, 'mode': Struct({'id': String, 'name': String}), 'frequency': Struct({'id': String, 'name': String})})),
        ('area', Struct({'id': String, 'name': String, 'url': String})),
        ('published_at', String),
        ('created_at', String),
        ('brand_snippet',
         Struct({'logo': Null, 'logo_xs': Null, 'logo_scalable': Struct({'default': Struct({'width': Int64, 'height': Int64, 'url': String}), 'xs': Struct({'width': Int64, 'height': Int64, 'url': String})}), 'picture': Null, 'picture_xs': Null, 'picture_scalable': Struct({'default': Struct({'width': Int64, 'height': Int64, 'url': String}), 'xs': Struct({'width': Int64, 'height': Int64, 'url': String})}), 'background': Struct({'color': String, 'gradient': Struct({'angle': Float64, 'color_list': List(Struct({'color': String, 'position': Float64}))})})})),
        ('url', 

In [47]:
# Полная схема (перевел Null -> String для безопасности, остальное как у вас)
vacancy_schema = {
    "response_letter_required": pl.Boolean,
    "salary_range": pl.Struct({
        "from": pl.Int64, "to": pl.Int64, "currency": pl.String, 
        "gross": pl.Boolean, 
        "mode": pl.Struct({"id": pl.String, "name": pl.String}), 
        "frequency": pl.Struct({"id": pl.String, "name": pl.String})
    }),
    "area": pl.Struct({"id": pl.String, "name": pl.String, "url": pl.String}),
    "published_at": pl.String,
    "created_at": pl.String,
    # brand_snippet: заменил Null на String во вложенных полях для безопасности
    "brand_snippet": pl.Struct({
        "logo": pl.String, 
        "logo_xs": pl.String, 
        "logo_scalable": pl.Struct({
            "default": pl.Struct({"width": pl.Int64, "height": pl.Int64, "url": pl.String}), 
            "xs": pl.Struct({"width": pl.Int64, "height": pl.Int64, "url": pl.String})
        }), 
        "picture": pl.String, 
        "picture_xs": pl.String, 
        "picture_scalable": pl.Struct({
            "default": pl.Struct({"width": pl.Int64, "height": pl.Int64, "url": pl.String}), 
            "xs": pl.Struct({"width": pl.Int64, "height": pl.Int64, "url": pl.String})
        }), 
        "background": pl.Struct({
            "color": pl.String, 
            "gradient": pl.Struct({
                "angle": pl.Float64, 
                "color_list": pl.List(pl.Struct({"color": pl.String, "position": pl.Float64}))
            })
        })
    }),
    "url": pl.String,
    "insider_interview": pl.Struct({"id": pl.String, "url": pl.String}),
    "working_time_intervals": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "work_format": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "night_shifts": pl.Boolean,
    "internship": pl.Boolean,
    "adv_context": pl.String, # Заменил Null на String
    "accept_incomplete_resumes": pl.Boolean,
    "branding": pl.Struct({"type": pl.String, "tariff": pl.String}),
    "professional_roles": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "apply_alternate_url": pl.String,
    "experience": pl.Struct({"id": pl.String, "name": pl.String}),
    "id": pl.String,
    "schedule": pl.Struct({"id": pl.String, "name": pl.String}),
    "employment": pl.Struct({"id": pl.String, "name": pl.String}),
    "show_contacts": pl.Boolean,
    "relations": pl.List(pl.String), # Заменил List(Null) на List(String)
    "fly_in_fly_out_duration": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "working_hours": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "name": pl.String,
    "accept_temporary": pl.Boolean,
    "has_test": pl.Boolean,
    "type": pl.Struct({"id": pl.String, "name": pl.String}),
    "alternate_url": pl.String,
    "working_days": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "snippet": pl.Struct({"requirement": pl.String, "responsibility": pl.String}),
    "employment_form": pl.Struct({"id": pl.String, "name": pl.String}),
    "response_url": pl.String,
    "is_adv_vacancy": pl.Boolean,
    "department": pl.Struct({"id": pl.String, "name": pl.String}),
    "sort_point_distance": pl.Float64, # Заменил Null на Float64 (обычно дистанция это число)
    "employer": pl.Struct({
        "id": pl.String, "name": pl.String, "url": pl.String, "alternate_url": pl.String, 
        "logo_urls": pl.Struct({"original": pl.String, "90": pl.String, "240": pl.String}), 
        "vacancies_url": pl.String, "country_id": pl.Int64, 
        "accredited_it_employer": pl.Boolean, "trusted": pl.Boolean
    }),
    "show_logo_in_search": pl.Boolean,
    "contacts": pl.Struct({"name": pl.String, "email": pl.String, "phones": pl.List(pl.String)}), # Null -> String
    "working_time_modes": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "salary": pl.Struct({"from": pl.Int64, "to": pl.Int64, "currency": pl.String, "gross": pl.Boolean}),
    "work_schedule_by_days": pl.List(pl.Struct({"id": pl.String, "name": pl.String})),
    "archived": pl.Boolean,
    "adv_response_url": pl.String, # Null -> String
    "address": pl.Struct({
        "city": pl.String, "street": pl.String, "building": pl.String, 
        "lat": pl.Float64, "lng": pl.Float64, "description": pl.String, "raw": pl.String, 
        "metro": pl.Struct({
            "station_name": pl.String, "line_name": pl.String, "station_id": pl.String, 
            "line_id": pl.String, "lat": pl.Float64, "lng": pl.Float64
        }), 
        "metro_stations": pl.List(pl.Struct({
            "station_name": pl.String, "line_name": pl.String, "station_id": pl.String, 
            "line_id": pl.String, "lat": pl.Float64, "lng": pl.Float64
        })), 
        "id": pl.String
    }),
    "premium": pl.Boolean,
    "video_vacancy": pl.Struct({
        "cover_picture": pl.Struct({"resized_path": pl.String, "resized_width": pl.Int64, "resized_height": pl.Int64}), 
        "snippet_picture": pl.Struct({"url": pl.String}), 
        "video": pl.Struct({"upload_id": pl.String, "url": pl.String}), 
        "snippet_video": pl.Struct({"upload_id": pl.String, "url": pl.String}), 
        "video_url": pl.String, 
        "snippet_video_url": pl.String, 
        "snippet_picture_url": pl.String
    }),
    # То самое поле, которое иногда отсутствует
    "immediate_redirect_url": pl.String 
}

In [48]:
# Читаем все файлы сразу с жесткой схемой
df_lazy = pl.scan_ndjson(
    "s3://bronze/hh/vacancies/date=2025-12-02/*.jsonl.gz",
    storage_options=storage_options,
    schema=vacancy_schema  # <--- Ключевой момент
)

# Проверяем
print(df_lazy.count().collect())

shape: (1, 49)
┌─────────────┬────────────┬────────┬────────────┬───┬─────────┬─────────┬────────────┬────────────┐
│ response_le ┆ salary_ran ┆ area   ┆ published_ ┆ … ┆ address ┆ premium ┆ video_vaca ┆ immediate_ │
│ tter_requir ┆ ge         ┆ ---    ┆ at         ┆   ┆ ---     ┆ ---     ┆ ncy        ┆ redirect_u │
│ ed          ┆ ---        ┆ u32    ┆ ---        ┆   ┆ u32     ┆ u32     ┆ ---        ┆ rl         │
│ ---         ┆ u32        ┆        ┆ u32        ┆   ┆         ┆         ┆ u32        ┆ ---        │
│ u32         ┆            ┆        ┆            ┆   ┆         ┆         ┆            ┆ u32        │
╞═════════════╪════════════╪════════╪════════════╪═══╪═════════╪═════════╪════════════╪════════════╡
│ 233167      ┆ 200906     ┆ 233167 ┆ 233167     ┆ … ┆ 192937  ┆ 233167  ┆ 1339       ┆ 704        │
└─────────────┴────────────┴────────┴────────────┴───┴─────────┴─────────┴────────────┴────────────┘
